In [7]:
import os
import cv2
import numpy as np
import requests
from tensorflow.keras.models import load_model

In [10]:
MODEL_URL = (
    "https://github.com/AikagraGupta/DetectIT/"
    "releases/download/v1.0.0/Meso4_DF_full.keras"
)
MODEL_PATH = os.path.join("weights", "Meso4_DF_full.keras")

if not os.path.exists(MODEL_PATH):
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
    print(f"Downloading model from {MODEL_URL} …")
    resp = requests.get(MODEL_URL, stream=True)
    resp.raise_for_status()
    with open(MODEL_PATH, "wb") as f:
        for chunk in resp.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print("Model downloaded.")

In [9]:
model = load_model(MODEL_PATH, compile=False)
print("Loaded Meso4 model.")

Loaded Meso4 model.


In [11]:
def predict_video(video_path: str,
                  model,
                  num_samples: int = 30,
                  target_size: tuple = (256, 256)):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        raise ValueError("Video contains no frames")

    interval = max(1, total_frames // num_samples)
    scores = []
    idx = 0
    taken = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if idx % interval == 0 and taken < num_samples:
            
            img = cv2.resize(frame, target_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.astype("float32") / 255.0
            img = np.expand_dims(img, 0)  # shape (1, H, W, 3)

            score = float(model.predict(img, verbose=0)[0][0])
            scores.append(score)
            taken += 1

        idx += 1

    cap.release()
    if not scores:
        raise ValueError("No frames sampled from video")

    avg_score = float(np.mean(scores))
    label = "real" if avg_score >= 0.5 else "deepfake"
    return label, avg_score, scores

In [12]:
video_file = "face4.mkv"
if not os.path.exists(video_file):
    raise FileNotFoundError(f"{video_file} not found (cwd={os.getcwd()})")

print(f"▶️ Processing video: {video_file}")
label, confidence, frame_scores = predict_video(video_file, model)
print(f"▶️ Video‐level verdict: {label.upper()} ({confidence*100:.1f}% confidence)")

▶️ Processing video: face4.mkv
▶️ Video‐level verdict: REAL (50.2% confidence)
